In [1]:
import numpy as np
import pandas as pd
from keras import models
from keras import layers
from keras import optimizers
from keras import losses 
from keras import metrics
import matplotlib.pyplot as plt

/home/tony/dnn/anaconda2/envs/myjupyter/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
training_set = pd.read_csv('./input/train.csv')
testing_set = pd.read_csv('./input/test.csv')

x_train = training_set.drop(['PassengerId','Name','Ticket','Survived'], axis=1)
y_train = training_set['Survived']
print("x_train:\n%r" % (x_train[:10]))

x_test = testing_set.drop(['PassengerId','Name','Ticket'], axis=1)

x_train:
   Pclass     Sex   Age  SibSp  Parch     Fare Cabin Embarked
0       3    male  22.0      1      0   7.2500   NaN        S
1       1  female  38.0      1      0  71.2833   C85        C
2       3  female  26.0      0      0   7.9250   NaN        S
3       1  female  35.0      1      0  53.1000  C123        S
4       3    male  35.0      0      0   8.0500   NaN        S
5       3    male   NaN      0      0   8.4583   NaN        Q
6       1    male  54.0      0      0  51.8625   E46        S
7       3    male   2.0      3      1  21.0750   NaN        S
8       3  female  27.0      0      2  11.1333   NaN        S
9       2  female  14.0      1      0  30.0708   NaN        C


In [3]:
x_train['Age'] = x_train['Age'].fillna(x_train['Age'].mean())
x_test['Age'] = x_test['Age'].fillna(x_test['Age'].mean())

In [4]:
def simplify_ages(df):
    #df['Age'] = df['Age'].fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior']
    categories = pd.cut(df['Age'], bins, labels=group_names)
    df['Age'] = categories.cat.codes 
    return df

def simplify_cabins(df):
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Cabin'] = df['Cabin'].apply(lambda x: x[0])
    df['Cabin'] =  pd.Categorical(df['Cabin'])
    df['Cabin'] = df['Cabin'].cat.codes 
    return df

def simplify_fares(df):
    df['Fare'] = df.Fare.fillna(-0.5)
    bins = (-1, 0, 8, 15, 31, 1000)
    group_names = ['Unknown', '1_quartile', '2_quartile', '3_quartile', '4_quartile']
    categories = pd.cut(df['Fare'], bins, labels=group_names)
    df['Fare'] = categories.cat.codes 
    return df

def simplify_sex(df):
    df['Sex'] = pd.Categorical(df['Sex'])
    df['Sex'] = df['Sex'].cat.codes 
    return df

def simplify_embarked(df):
    df['Embarked'] = pd.Categorical(df['Embarked'])
    df['Embarked'] = df['Embarked'].cat.codes + 1
    return df

def transform_features(df):
    df = simplify_ages(df)
    df = simplify_cabins(df)
    df = simplify_fares(df)
    df = simplify_sex(df)
    df = simplify_embarked(df)
    return df

In [5]:
transform_features(x_train)
transform_features(x_test)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,5,0,0,1,7,2
1,3,0,6,1,0,1,7,3
2,2,1,7,0,0,2,7,2
3,3,1,5,0,0,2,7,3
4,3,0,4,1,1,2,7,3
5,3,1,3,0,0,2,7,3
6,3,0,5,0,0,1,7,2
7,2,1,5,1,1,3,7,3
8,3,0,3,0,0,1,7,1
9,3,1,4,2,0,3,7,3


In [6]:
model = models.Sequential()
model.add(layers.Dense(32, activation='relu', 
                       input_shape=(8,)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
#model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

In [8]:
y_train = np.asarray(y_train)
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)

validation_size = 200

x_val = x_train[:validation_size]
partial_x_train = x_train[validation_size:]

y_val = y_train[:validation_size]
partial_y_train = y_train[validation_size:]

In [11]:
history = model.fit(partial_x_train, partial_y_train, epochs=300, validation_data=(x_val, y_val))

Train on 691 samples, validate on 200 samples
Epoch 1/300
691/691 [==============================] - 0s 69us/step - loss: 0.3349 - binary_accuracy: 0.8596 - val_loss: 0.4817 - val_binary_accuracy: 0.7800
Epoch 2/300
691/691 [==============================] - 0s 68us/step - loss: 0.3407 - binary_accuracy: 0.8611 - val_loss: 0.4809 - val_binary_accuracy: 0.7900
Epoch 3/300
691/691 [==============================] - 0s 66us/step - loss: 0.3334 - binary_accuracy: 0.8553 - val_loss: 0.4610 - val_binary_accuracy: 0.8150
Epoch 4/300
691/691 [==============================] - 0s 62us/step - loss: 0.3413 - binary_accuracy: 0.8582 - val_loss: 0.5781 - val_binary_accuracy: 0.7450
Epoch 5/300
691/691 [==============================] - 0s 66us/step - loss: 0.3345 - binary_accuracy: 0.8596 - val_loss: 0.4646 - val_binary_accuracy: 0.8000
Epoch 6/300
691/691 [==============================] - 0s 61us/step - loss: 0.3357 - binary_accuracy: 0.8625 - val_loss: 0.5075 - val_binary_accuracy: 0.7850
Epoch 

691/691 [==============================] - 0s 71us/step - loss: 0.3205 - binary_accuracy: 0.8770 - val_loss: 0.4963 - val_binary_accuracy: 0.8150
Epoch 53/300
691/691 [==============================] - 0s 70us/step - loss: 0.3192 - binary_accuracy: 0.8611 - val_loss: 0.5220 - val_binary_accuracy: 0.7550
Epoch 54/300
691/691 [==============================] - 0s 76us/step - loss: 0.3143 - binary_accuracy: 0.8813 - val_loss: 0.5020 - val_binary_accuracy: 0.8000
Epoch 55/300
691/691 [==============================] - 0s 70us/step - loss: 0.3214 - binary_accuracy: 0.8582 - val_loss: 0.5142 - val_binary_accuracy: 0.7950
Epoch 56/300
691/691 [==============================] - 0s 70us/step - loss: 0.3202 - binary_accuracy: 0.8712 - val_loss: 0.5471 - val_binary_accuracy: 0.7550
Epoch 57/300
691/691 [==============================] - 0s 71us/step - loss: 0.3106 - binary_accuracy: 0.8741 - val_loss: 0.5718 - val_binary_accuracy: 0.7700
Epoch 58/300
691/691 [==============================] - 0s 

691/691 [==============================] - 0s 70us/step - loss: 0.3031 - binary_accuracy: 0.8784 - val_loss: 0.5783 - val_binary_accuracy: 0.7650
Epoch 104/300
691/691 [==============================] - 0s 66us/step - loss: 0.3086 - binary_accuracy: 0.8799 - val_loss: 0.5439 - val_binary_accuracy: 0.7850
Epoch 105/300
691/691 [==============================] - 0s 69us/step - loss: 0.3084 - binary_accuracy: 0.8669 - val_loss: 0.5658 - val_binary_accuracy: 0.7700
Epoch 106/300
691/691 [==============================] - 0s 63us/step - loss: 0.3079 - binary_accuracy: 0.8726 - val_loss: 0.5349 - val_binary_accuracy: 0.7700
Epoch 107/300
691/691 [==============================] - 0s 61us/step - loss: 0.2985 - binary_accuracy: 0.8857 - val_loss: 0.5365 - val_binary_accuracy: 0.7900
Epoch 108/300
691/691 [==============================] - 0s 61us/step - loss: 0.3005 - binary_accuracy: 0.8828 - val_loss: 0.5615 - val_binary_accuracy: 0.8050
Epoch 109/300
691/691 [==============================]

Epoch 154/300
691/691 [==============================] - 0s 74us/step - loss: 0.2952 - binary_accuracy: 0.8770 - val_loss: 0.5657 - val_binary_accuracy: 0.7600
Epoch 155/300
691/691 [==============================] - 0s 65us/step - loss: 0.3022 - binary_accuracy: 0.8755 - val_loss: 0.5319 - val_binary_accuracy: 0.7850
Epoch 156/300
691/691 [==============================] - 0s 61us/step - loss: 0.2958 - binary_accuracy: 0.8741 - val_loss: 0.5605 - val_binary_accuracy: 0.7550
Epoch 157/300
691/691 [==============================] - 0s 69us/step - loss: 0.3050 - binary_accuracy: 0.8770 - val_loss: 0.5766 - val_binary_accuracy: 0.7700
Epoch 158/300
691/691 [==============================] - 0s 69us/step - loss: 0.3012 - binary_accuracy: 0.8770 - val_loss: 0.5314 - val_binary_accuracy: 0.7950
Epoch 159/300
691/691 [==============================] - 0s 75us/step - loss: 0.2953 - binary_accuracy: 0.8784 - val_loss: 0.5548 - val_binary_accuracy: 0.7750
Epoch 160/300
691/691 [=================

691/691 [==============================] - 0s 66us/step - loss: 0.2830 - binary_accuracy: 0.8842 - val_loss: 0.5448 - val_binary_accuracy: 0.7850
Epoch 206/300
691/691 [==============================] - 0s 60us/step - loss: 0.2889 - binary_accuracy: 0.8828 - val_loss: 0.5484 - val_binary_accuracy: 0.7550
Epoch 207/300
691/691 [==============================] - 0s 60us/step - loss: 0.2875 - binary_accuracy: 0.8741 - val_loss: 0.5486 - val_binary_accuracy: 0.7550
Epoch 208/300
691/691 [==============================] - 0s 60us/step - loss: 0.2863 - binary_accuracy: 0.8784 - val_loss: 0.5945 - val_binary_accuracy: 0.7750
Epoch 209/300
691/691 [==============================] - 0s 61us/step - loss: 0.2894 - binary_accuracy: 0.8813 - val_loss: 0.5752 - val_binary_accuracy: 0.7700
Epoch 210/300
691/691 [==============================] - 0s 69us/step - loss: 0.2879 - binary_accuracy: 0.8770 - val_loss: 0.6381 - val_binary_accuracy: 0.7700
Epoch 211/300
691/691 [==============================]

691/691 [==============================] - 0s 61us/step - loss: 0.2745 - binary_accuracy: 0.8929 - val_loss: 0.5832 - val_binary_accuracy: 0.7700
Epoch 257/300
691/691 [==============================] - 0s 62us/step - loss: 0.2780 - binary_accuracy: 0.8871 - val_loss: 0.5977 - val_binary_accuracy: 0.7800
Epoch 258/300
691/691 [==============================] - 0s 61us/step - loss: 0.2812 - binary_accuracy: 0.8842 - val_loss: 0.5196 - val_binary_accuracy: 0.7900
Epoch 259/300
691/691 [==============================] - 0s 67us/step - loss: 0.2783 - binary_accuracy: 0.8799 - val_loss: 0.5924 - val_binary_accuracy: 0.8000
Epoch 260/300
691/691 [==============================] - 0s 64us/step - loss: 0.2806 - binary_accuracy: 0.8944 - val_loss: 0.6150 - val_binary_accuracy: 0.7700
Epoch 261/300
691/691 [==============================] - 0s 61us/step - loss: 0.2708 - binary_accuracy: 0.8958 - val_loss: 0.6450 - val_binary_accuracy: 0.7650
Epoch 262/300
691/691 [==============================]